# HR Applicant Filtering System

## Data Collection

In [1]:
#@title Installation
%%capture
import importlib
# List of required libraries
required_libraries = ["PyMuPDF", "nltk", "python-docx"]

# Check if each library is installed, and install if not
for lib in required_libraries:
    try:
        importlib.import_module(lib)
        print(f"{lib} is already installed.")
    except ImportError:
        print(f"{lib} is not installed. Installing...")
        !pip install {lib}

In [2]:
#@title Libraries
%%capture
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
from IPython.display import FileLink
import gdown  # For downloading files from Google Drive
from bs4 import BeautifulSoup  # For parsing HTML
from googleapiclient.http import MediaIoBaseDownload
from datetime import datetime, timezone, timedelta
from collections import Counter
from IPython.display import display, HTML
import ipywidgets as widgets
from io import BytesIO
import ipywidgets as widgets
import gspread
from google.colab import auth
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2 import service_account
from googleapiclient.discovery import build
import json
import requests
import pandas as pd
import numpy as np
import openpyxl
import re
import os
import fitz
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from docx import Document ;

In [3]:
# @title BLK
json_key = {

}

In [4]:
# @title Authentication/Access
# Authenticate with Google Sheets using the JSON key
scope_sheets = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds_sheets = ServiceAccountCredentials.from_json_keyfile_dict(json_key, scope_sheets)
client_sheets = gspread.authorize(creds_sheets)

# Authenticate with Google Drive using the JSON key
scope_drive = ['https://www.googleapis.com/auth/drive']
creds_drive = service_account.Credentials.from_service_account_info(json_key, scopes=scope_drive)
service_drive = build('drive', 'v3', credentials=creds_drive)

# Access Google Sheets
worksheet_name = "Sheet1"
worksheet = client_sheets.open('screening_repo').worksheet(worksheet_name)

In [5]:
# @title Appending the Dataframe
# Check if DataFrame 'df' already exists
if 'df' in globals():
    print("DataFrame 'df' already exists. Skipping...")
else:
    # Get all values from the worksheet
    rows = worksheet.get_all_values()

    # Convert to DataFrame
    df = pd.DataFrame(rows[1:], columns=rows[0])  # Assuming the first row contains column headers
    df['Total Score'] = 0
    df['Qualification'] = ''
    print("DataFrame 'df' created successfully.")

DataFrame 'df' created successfully.


In [6]:
# @title DFB
# Access Google Sheets
worksheet_name = "TS"
worksheet2 = client_sheets.open('Initial Screening').worksheet(worksheet_name)


# Get all values from the worksheet
rows = worksheet2.get_all_values()

    # Convert to DataFrame
dfb = pd.DataFrame(rows[1:], columns=rows[0])  # Assuming the first row contains column headers

print("DataFrame 'dfb' created successfully.")

DataFrame 'dfb' created successfully.


In [7]:
# @title Renaming columns
if "CV_Link" in dfb.columns:
    print("CV_Link already exists. Skipping column renaming...")
else:
    # Rename columns by index
    dfb.columns.values[1] = 'Name'
    dfb.columns.values[2] = 'Position'
    dfb.columns.values[4] = 'Contact_No'
    dfb.columns.values[5] = 'Email'
    dfb.columns.values[7] = 'Employment_Status'
    dfb.columns.values[9] = 'Training_MF_5AM'
    dfb.columns.values[10] = 'Device'
    dfb.columns.values[11] = 'ISP'
    dfb.columns.values[12] = 'Jobsite/Refrerral'
    dfb.columns.values[14] = 'CV_Link'
    print("Columns renamed successfully.")

# Remove leading and trailing whitespace from column names
dfb.columns = dfb.columns.str.strip()


Columns renamed successfully.


In [ ]:
# @title Appending New Rows
# Left merge dataframes dfb and df on the "CV_Link" column
merged_df = pd.merge(dfb, df[['CV_Link', 'Total Score', 'Qualification', 'Text', 'CleanedText']],
                     on='CV_Link', how='left')

# Fill NaN values with the desired values
merged_df['Total Score'].fillna(0, inplace=True)
merged_df['Qualification'].fillna('', inplace=True)
merged_df['Text'].fillna('', inplace=True)
merged_df['CleanedText'].fillna('', inplace=True)

# Drop rows with empty values in the "CV_Link" column
merged_df.dropna(subset=['CV_Link'], inplace=True)

# Filter out rows with empty strings in the "CV_Link" column
merged_df = merged_df[merged_df['CV_Link'] != '']

# Drop rows in the "Status" column that contain strings
merged_df = merged_df[~merged_df['Screening  Status'].str.contains('[a-zA-Z]')]

# Reset index
merged_df.reset_index(drop=True, inplace=True)


merged_df.tail()

In [10]:
# @title GDrive Engine
# Define the function for batch processing to download and extract text
def batch_download_and_extract_text(service, urls):
    texts = []
    for url in urls:
        try:
            # Extract file ID from the URL
            file_id = url.split('=')[-1]

            # Retrieve metadata to get the file name
            file_metadata = service.files().get(fileId=file_id, fields="name").execute()
            filename = file_metadata['name']

            # Download the file
            request = service.files().get_media(fileId=file_id)
            fh = BytesIO()
            downloader = MediaIoBaseDownload(fh, request)

            done = False
            while not done:
                status, done = downloader.next_chunk()

            fh.seek(0)

            # Extract text based on file type
            _, file_extension = os.path.splitext(filename)
            if file_extension.lower() == '.pdf':
                # Extract text from the PDF file using PyMuPDF (fitz)
                text = ""
                with fitz.open(stream=fh, filetype="pdf") as pdf_doc:
                    for page_num in range(len(pdf_doc)):
                        page = pdf_doc.load_page(page_num)
                        text += page.get_text()
            elif file_extension.lower() in ['.docx', '.doc']:
                # Extract text from the DOCX or DOC file using python-docx
                doc = Document(fh)
                text = "\n".join([para.text for para in doc.paragraphs])
            else:
                # Unsupported file type
                print(f"Unsupported file type for URL: {url}")
                text = "Invalid File Format/No Access"

            if not text.strip():
                text = "Invalid File Format/No Access"

            texts.append(text)
        except Exception as e:
            print(f"An error occurred while downloading or extracting text from the file: {e}")
            texts.append("Invalid File Format/No Access")

    return texts

In [11]:
# @title Reading the CVs

# Check if there is any empty string in the "CleanedText" column
if (merged_df["CleanedText"].str.strip() == '').any():
    # Filter the DataFrame to get only the rows with empty strings in the "CleanedText" column
    filtered_df = merged_df[merged_df["CleanedText"].str.strip() == '']

    # Extract the URLs from the filtered DataFrame
    urls = filtered_df["CV_Link"].tolist()

    # Use ThreadPoolExecutor to perform parallel processing
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Submit tasks for batch processing
        future = executor.submit(batch_download_and_extract_text, service_drive, urls)
        texts = future.result()

    # Assign extracted text to the "Text" column in the filtered DataFrame
    filtered_df["Text"] = texts

    # Update the original DataFrame with the updated values from the filtered DataFrame
    merged_df.update(filtered_df)

else:
    # Print all updated DataFrame
    print("All Updated")


All Updated


## Applicant Shortlisting Tool

#### Instructions:

1. Input the necessary information such as <font color="orange"><b>Job Title</b></font>, <font color="green"><b>Job Description</b></font>, <font color="red"><b>Qualifying Keywords</b></font>, and <font color="yellow"><b>Supplementary Keywords</b></font>  in the designated fields below.

2. Press **Connect** at the upper right corner. <font color="green"><b>Wait for a green check mark to appear</b></font>.

3. Click **Runtime** dropdown and click **Run All** to run the notebook. A prompt will appear but just click "Run Anyway". Each run takes about less than 2 minutes.

4. You can change your entries in the Job Title, Job Description, etc. fields and run it again by following **Step 3**.

4. Scroll down to view the shortlist at the bottom. You can manipulate it to show different Job Positions by clicking on the "**Select Position**" dropdown button. Other filters are also available.

5. Scroll sideward to reveal more information about the applicants. Can now also be viewed in Fullscreen (See documentation).

6. You can also export your chosen shortlist into an Excel file by clicking the "**Export as Excel File**" button. It is at the bottom of the shortlist.

7. Click the little Folder icon on the left side to open up the window pane. Download the file by right clicking and choosing download.

9. If an error pops-up. Refresh the page itself. Refreshing the page would need you to enter the necessary fields again.

10. For persistent errors, contact the notebook owner.

In [12]:
#@title <font color='Orange'>Job Title </font>
job_post = ""# @param {type: "string"}

In [13]:
#@title <font color='Green'>Job Description </font>
user_input = ""# @param {type: "string"}

In [14]:
#@title  <font color='Red'> Qualifying Keywords</font>
Qualifying_keywords = ""# @param {type: "string"}


In [15]:
#@title <font color='yellow'>  Supplementary Keywords </font>
supplementary_keywords = ""# @param {type: "string"}


## Processing

In [16]:
#@title Tokenization


# Define the list of negative words
negative_words = ["degree", "proficiency", "exceptional", "proven", "experience", "strong", "skills", "city", "university",
                  "2023","2024", "management", "2022", "customer", "school", "philippines",
                  "ability", "interpret", "complex", "information", "excellent",
                  "work", "effectively", "proficiency", "utilizing", "various",
                  "conduct", "in-depth", "research", "across", "diverse", "sources",
                  "gather", "relevant", "data", "evaluate", "credibility", "relevancy",
                  "obtained", "different", "translate", "synthesize", "comprehensive",
                  "english", "reports", "maintaining", "accuracy", "clarity",
                  "decision-making", "processes", "stay", "updated", "changes",
                  "global", "markets", "ensure", "accuracy", "relevance", "reports",
                  "utilize", "enhance", "analysis", "processes", "knowledge",
                  "familiarity", "understands", "intricacies", "platforms",
                  "familiarity", "identify", "filter", "irrelevant", "posts",
                  "assess", "summarize", "comments", "made", "posts", "access", "file", "invalid",
                  "group", "structure", "predefined", "criteria", "topics",
                  "good", "eg", "etc", "Main", "task", "tasks", 'include', 'required', 'skills', 'other']

# Check if there is any empty string in the "CleanedText" column
if (merged_df["CleanedText"].str.strip() == '').any():
    # Filter the DataFrame to get only the rows with empty strings in the "CleanedText" column
    filtered_df2 = merged_df[merged_df["CleanedText"].str.strip() == '']

    # Define function to remove numbers from text
    def remove_numbers(text):
        if isinstance(text, str):
            return re.sub(r'\d+', '', text)
        else:
            return str(text)

    # Apply text cleaning functions to the 'Text' and 'Specify your skills relevant to the position you are applying for.' columns
    filtered_df2['CleanedText'] = filtered_df2['Text'].apply(lambda x: ', '.join([word for word in word_tokenize(x.lower()) if len(word) > 1 and word.isalnum() and word.lower() not in stopwords.words('english')]))
    filtered_df2['CleanedSkills'] = filtered_df2['Specify your skills relevant to the position you are applying for.'].apply(lambda x: ', '.join([word for word in word_tokenize(x.lower()) if len(word) > 1 and word.isalnum() and word.lower() not in stopwords.words('english')]))

    # Remove negative words from CleanedText
    filtered_df2['CleanedText'] = filtered_df2['CleanedText'].apply(lambda x: ' '.join([word for word in word_tokenize(x.lower()) if word.lower() not in negative_words]))

    # Combine CleanedText and CleanedSkills, remove duplicates, and join back into a string
    filtered_df2['CleanedText'] = (filtered_df2['CleanedText'] + ', ' + filtered_df2['CleanedSkills']).apply(lambda x: ', '.join(set(word_tokenize(x))))

    # Update the merged_df with the cleaned data
    merged_df.update(filtered_df2)

else:
    # Print all updated DataFrame
    print("All Updated")

All Updated


In [17]:
#@title Process 1

input_tokens = set(word_tokenize(user_input.lower()))

pd2 = pd.DataFrame({"UserInput": list(input_tokens)})

pd2 = pd.merge(pd2, pd.DataFrame({"NegativeWords": negative_words}), how="left", left_on="UserInput", right_on="NegativeWords")

pd2_cleaned = pd2[pd2['NegativeWords'].isna()].drop(columns=['NegativeWords'])

cleaned_list = ', '.join(pd2_cleaned['UserInput'])
cleaned_list = re.sub(r'[^a-zA-Z0-9\s]', '', cleaned_list)
cleaned_tokens = word_tokenize(cleaned_list)

stop_words = set(stopwords.words('english'))
cleaned_tokens = [word for word in cleaned_tokens if word.lower() not in stop_words]

df2 = pd.DataFrame({"CleanedList": [', '.join(cleaned_tokens)]})

In [18]:
#@title Process 2


supplementary_tokens = set(word_tokenize(supplementary_keywords.lower()))

df3 = pd.DataFrame({"SupplementaryKeywords": list(supplementary_tokens)})

df3['SupplementaryKeywords'] = df3['SupplementaryKeywords'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))

stop_words = set(stopwords.words('english'))
df3['SupplementaryKeywords'] = df3['SupplementaryKeywords'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word.lower() not in stop_words]))


In [19]:
#@title Process 3

Qualifying_tokens = set(word_tokenize(Qualifying_keywords.lower()))


df4 = pd.DataFrame({"QualifyingKeywords": list(Qualifying_tokens)})

df4['QualifyingKeywords'] = df4['QualifyingKeywords'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))

stop_words = set(stopwords.words('english'))
df4['QualifyingKeywords'] = df4['QualifyingKeywords'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word.lower() not in stop_words]))


In [20]:
#@title Scoring


def calculate_score(row):
    total_score = 0

    cleaned_text_tokens = set(row['CleanedText'].split(', '))

    if 'CleanedList' in df2.columns and df2['CleanedList'].dtype == 'object':
        total_score += len(set(cleaned_text_tokens) & set(df2['CleanedList'].str.split(', ').explode().unique()))

    if 'SupplementaryKeywords' in df3.columns and df3['SupplementaryKeywords'].dtype == 'object':
        total_score += len(set(cleaned_text_tokens) & set(df3['SupplementaryKeywords'].str.split(', ').explode().unique()))

    if 'QualifyingKeywords' in df4.columns and df4['QualifyingKeywords'].dtype == 'object':
        total_score += len(set(cleaned_text_tokens) & set(df4['QualifyingKeywords'].str.split(', ').explode().unique()))

    return total_score

merged_df['Total Score'] = merged_df.apply(calculate_score, axis=1)

In [21]:
#@title Qualification

def determine_qualification(row):
    cleaned_text_tokens = set(row['CleanedText'].split(', '))

    if 'QualifyingKeywords' in df4.columns and df4['QualifyingKeywords'].dtype == 'object':
        qualifying_keywords_tokens = set(df4['QualifyingKeywords'].str.split(', ').explode().unique())

        if set(cleaned_text_tokens) & qualifying_keywords_tokens:
            return "Qualified"
        else:
            return "Not Qualified"
    else:
        return "N/A"

merged_df['Qualification'] = merged_df.apply(determine_qualification, axis=1)


In [22]:
#@title Computation

qualified_df = merged_df[merged_df['Qualification'] == 'Qualified']

qualified_df = qualified_df.sort_values(by='Total Score', ascending=False)

not_qualified_df = merged_df[merged_df['Qualification'].isin(['N/A', 'Not Qualified'])]

not_qualified_df = not_qualified_df.sort_values(by='Total Score', ascending=False)

In [23]:
# @title Saving

# Drop columns with empty names
merged_df = merged_df.drop(columns=['' for col in merged_df.columns if col == ''])

# Open the existing Google Sheet named "screening_repo"
sheet3 = client_sheets.open('screening_repo')

# Select the first (default) worksheet
worksheet3 = sheet3.get_worksheet(0)

# Clear the contents of the worksheet
worksheet3.clear()

# Get the column headers
headers = merged_df.columns.tolist()

# Convert DataFrame to a list of lists, including headers
values_with_headers = [headers] + merged_df.fillna('').values.tolist()

# Update the values in the worksheet
worksheet3.update('A1', values_with_headers)

<ipython-input-23-5af4d3cc4d3e>:22: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet3.update('A1', values_with_headers)


{'spreadsheetId': '1PS782vFp9eXtSKkkknre4-XzM0_XJoWXER7zPwaxNNw',
 'updatedRange': 'Sheet1!A1:Z445',
 'updatedRows': 445,
 'updatedColumns': 26,
 'updatedCells': 11570}

## Shortlist

In [24]:
#@title Filtered Applicants

# Convert "Timestamp" column to datetime
merged_df['Timestamp'] = pd.to_datetime(merged_df['Timestamp'], format='%d/%m/%Y %H:%M:%S')

# Format months in "Timestamp" column as month names
merged_df['Month'] = merged_df['Timestamp'].dt.strftime('%B')  # '%B' gives full month name

# Columns to hide from the output
columns_to_hide = ["Text", "CleanedText", "Specify your skills relevant to the position you are applying for."]

# Default values for dropdowns
def set_default_values():
    default_employment_status = "None"
    default_training = "Yes"
    default_year = merged_df['Timestamp'].dt.year.max()

    employment_status_dropdown.value = default_employment_status
    training_dropdown.value = default_training
    year_dropdown.value = default_year

# Create dropdown widgets for unique positions
unique_positions = merged_df["Position"].unique()
position_dropdown = widgets.Dropdown(options=unique_positions, description='Select Position:')

# Custom display function to format the DataFrame output
def update_dropdowns(*args):
    try:
        filtered_df = merged_df[merged_df["Position"] == position_dropdown.value]
        unique_statuses = filtered_df["Employment_Status"].unique()
        unique_trainings = filtered_df["Training_MF_5AM"].unique()
        unique_years = filtered_df['Timestamp'].dt.year.unique()
        unique_months = filtered_df['Month'].unique()

        employment_status_dropdown.options = unique_statuses
        training_dropdown.options = unique_trainings
        year_dropdown.options = unique_years
        month_dropdown.options = unique_months

        set_default_values()
    except Exception as e:
        pass  # Suppress any errors

# Link dropdown widgets to update function
position_dropdown.observe(update_dropdowns, 'value')

# Create dropdown widgets for dependent options
employment_status_dropdown = widgets.Dropdown(description='Employment Status:')
training_dropdown = widgets.Dropdown(description='Training MF 5AM:')
year_dropdown = widgets.Dropdown(description='Select Year:')
month_dropdown = widgets.Dropdown(description='Select Month:')

# Dropdown for Total Score filter
score_filter_dropdown = widgets.Dropdown(options=["Above or Equal to 10", "Below 10"], description='Total Score Filter:')

# Call update function initially to set initial options
update_dropdowns()

# Specify columns to hide from the output
columns_to_hide = ["Text", "CleanedText", "Specify your skills relevant to the position you are applying for."]

# Custom display function to format the DataFrame output
def display_dataframe(position, employment_status, training, year, month, score_filter):
    try:
        filtered_df = merged_df[(merged_df["Position"] == position) &
                                (merged_df["Employment_Status"] == employment_status) &
                                (merged_df["Training_MF_5AM"] == training) &
                                (merged_df['Timestamp'].dt.year == year) &
                                (merged_df['Month'] == month)]

        if score_filter == "Above or Equal to 10":
            filtered_df = filtered_df[filtered_df['Total Score'] >= 10]
        elif score_filter == "Below 10":
            filtered_df = filtered_df[filtered_df['Total Score'] < 10]

        if filtered_df.empty:
            display(HTML("<p style='font-size:20px; font-weight:bold;'>No applicants meet the criteria.</p>"))
            return

        sorted_df = filtered_df.sort_values(by="Total Score", ascending=False)
        total_score = sorted_df.pop("Total Score")
        qualification = sorted_df.pop("Qualification")

        display_df = sorted_df.drop(columns_to_hide, axis=1)
        display_df.insert(0, "Total Score", total_score)
        display_df.insert(1, "Qualification", qualification)

        display_df = display_df.reindex(columns=["Position",
                                                 "Name",
                                                 "Qualification",
                                                 "Total Score",
                                                 "Employment_Status",
                                                 "Training_MF_5AM",
                                                 "CV_Link",
                                                 "Address",
                                                 "Contact_No",
                                                 "Email",
                                                 "Device",
                                                 "ISP",
                                                 "Birth Date",
                                                 "Timestamp"])

        top10 = display_df.head(10)

        html_output = top10.to_html(index=False)
        styled_html_output = f'<div style="overflow-x:auto;"><style>.dataframe tr {{ height: 25px; }}</style>{html_output}</div>'
        display(HTML(styled_html_output))

        export_button = widgets.Button(description="Export as Excel File")
        display(export_button)

        def export_as_excel_file(_):
            ph_time = timezone(timedelta(hours=8))
            current_datetime = datetime.now(ph_time).strftime("%Y-%m-%d_%H-%M-%S")

            directory = "/content/"
            filename = f"{position.replace('/', '_')}_{employment_status}_{training}_{current_datetime}.xlsx"
            filepath = os.path.join(directory, filename)

            top10.to_excel(filepath, index=False)

        export_button.on_click(export_as_excel_file)
    except Exception as e:
        pass  # Suppress any errors

# Link dropdown widgets to the display_dataframe function
widgets.interact(display_dataframe, position=position_dropdown, employment_status=employment_status_dropdown,
                 training=training_dropdown, year=year_dropdown, month=month_dropdown, score_filter=score_filter_dropdown)

interactive(children=(Dropdown(description='Select Position:', options=('Research Analyst/Writer (Part-time)',…

<function __main__.display_dataframe(position, employment_status, training, year, month, score_filter)>